GROUP MEMBERS :
1.  Aniruddh Shukla 
2. Gaurav Singhal 
3. Himanshi Bajaj 


# Low-level implementation of a RNN

In [ ]:
import tensorflow as tf
import numpy as np
import os

In [ ]:
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
!python prepare_data.py shakespeare_input.txt skp

2020-05-25 17:19:03.360252: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 22981 sequences...
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
Serialized 2000 sequences...
Serialized 2100 sequences...
Serialized 2200 sequences...
Serialized 2300 sequences...
Serialized 2400 sequences...
Serialized 2500 sequences...
Serialized 2600 sequences...
Serialized 2700 sequences...
Serialized 2800 sequences...
Serialized 2900

In [ ]:
from prepare_data import parse_seq
import pickle

# this is just a datasets of "bytes" (not understandable)
data = tf.data.TFRecordDataset("skp.tfrecords")

# this maps a parser function that properly interprets the bytes over the dataset
# (with fixed sequence length 200)
# if you change the sequence length in preprocessing you also need to change it here
data = data.map(lambda x: parse_seq(x, 200))

# a map from characters to indices
vocab = pickle.load(open("skp_vocab", mode="rb"))
vocab_size = len(vocab)
# inverse mapping: indices to characters
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab)
print(vocab_size)

{'D': 1, 'C': 2, 'c': 3, 'q': 4, '&': 5, 'N': 6, '\n': 7, 'y': 8, '-': 9, '$': 10, 'F': 11, '!': 12, 'j': 13, '3': 14, 'G': 15, 'S': 16, 'W': 17, 'Z': 18, "'": 19, 'B': 20, 's': 21, 'e': 22, 'a': 23, ':': 24, 'z': 25, 'v': 26, 'U': 27, ']': 28, 't': 29, 'x': 30, 'V': 31, 'Q': 32, 'X': 33, 'n': 34, 'b': 35, 'R': 36, 'L': 37, 'r': 38, 'O': 39, 'I': 40, ' ': 41, 'o': 42, 'i': 43, 'g': 44, 'E': 45, 'l': 46, 'J': 47, 'd': 48, 'T': 49, '.': 50, 'w': 51, 'k': 52, 'f': 53, '?': 54, ',': 55, 'H': 56, ';': 57, '[': 58, 'P': 59, 'Y': 60, 'p': 61, 'u': 62, 'A': 63, 'K': 64, 'M': 65, 'm': 66, 'h': 67, '<S>': 0}
68


In [ ]:
def initialization(hidden_units):
  # Assuming hidden_units = 512 and Vocab_size = 68
  # 512*68 -> Weight b/w input and hidden state 
  W_xh = tf.Variable(np.random.uniform(size=(hidden_units, vocab_size), low=-.01, high=.01).astype(np.float32), trainable=True)
  # 512*512 -> Weight b/w hidden and hidden state
  W_hh = tf.Variable(np.random.uniform(size=(hidden_units, hidden_units), low=-.01, high=.01).astype(np.float32), trainable=True)
  # 512*68 -> Weight b/w hidden state and output
  W_ho = tf.Variable(np.random.uniform(size=(hidden_units,vocab_size), low=-.01, high=.01).astype(np.float32), trainable=True)
  
  # 512*1 -> Bias for hidden state
  b_h = tf.Variable(np.zeros((hidden_units, 1)), dtype=np.float32, trainable=True)
  # 1*68 -> Bias for output state
  b_o = tf.Variable(np.zeros((1,vocab_size)), dtype=np.float32, trainable=True)

  params = {}
  params['W_xh'] = W_xh
  params['W_hh'] = W_hh
  params['W_ho'] = W_ho
  params['b_h'] = b_h
  params['b_o'] = b_o
  return params

In [ ]:
def rnn_cell(h_prev, x_t, params):
  W_xh = params['W_xh']
  W_hh = params['W_hh']
  W_ho = params['W_ho']
  b_h = params['b_h']
  b_o = params['b_o']

  # Use tanh and sigmoid respectively.
  m1 = tf.matmul(W_xh, tf.transpose(x_t))
  m2 = tf.matmul(W_hh, h_prev)
  h_next = tf.tanh(m1 + m2 + b_h)
  logits = tf.matmul(tf.transpose(h_next),W_ho) + b_o
  out_t = tf.nn.softmax(logits,axis=1)
  # print("{} x {}".format(h_next.shape, out_t.shape))
  return h_next, logits, out_t 

In [ ]:
def loss_function(output,logits):
   xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits , labels=output))
   return xent

In [ ]:
def one_hot_encoding(x):
  return tf.one_hot(x, depth = vocab_size)

In [ ]:
BATCH_SIZE = 128
one_hot_encoded_data = data.map(one_hot_encoding)
train_data = one_hot_encoded_data.batch(batch_size=BATCH_SIZE,drop_remainder=True)
for x in train_data:
  print(x.shape)
  break

(128, 200, 68)


In [ ]:
label_data = data.batch(batch_size=BATCH_SIZE,drop_remainder=True)
for y in label_data:
  print(y.shape)
  break

(128, 200)


In [ ]:
def model_execution(epochs, params, optimizer,train_data,label_data):
  h_o =  tf.zeros((params['W_hh'].shape[0],BATCH_SIZE))
  for epoch in range(epochs):
    for (x_batch,y_batch) in zip(train_data,label_data):
      loss_per_batch = []
      with tf.GradientTape() as tape:
        for time_step in range(199):
          # coverting data into 128*68
          x_pass = x_batch[:,time_step,:]
          # 128
          label_pass = y_batch[:,time_step+1]

          h_next, logits, out_train = rnn_cell(h_o,x_pass,params)
         
          loss_in_each_time_step = loss_function(label_pass,logits)
          loss_per_batch.append(loss_in_each_time_step)

          h_o = h_next
          
          
        average_loss = tf.math.reduce_mean(loss_per_batch)
      grads = tape.gradient(average_loss, list(params.values()))
      optimizer.apply_gradients(zip(grads, list(params.values())))
    print("Epoch: {}, Loss: {}".format(epoch, average_loss))


In [ ]:
# Hyper-parameters
hidden_units = 512
epochs = 40
learning_rate = 0.001

# Getting initialization parameters
params = initialization(hidden_units)
# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model_execution(epochs, params, optimizer,train_data,label_data)

hi from params
Epoch: 0, Loss: 3.100139617919922
Epoch: 1, Loss: 2.5197675228118896
Epoch: 2, Loss: 2.278074264526367
Epoch: 3, Loss: 2.137728691101074
Epoch: 4, Loss: 2.046231746673584
Epoch: 5, Loss: 1.968592643737793
Epoch: 6, Loss: 1.9000571966171265
Epoch: 7, Loss: 1.8412927389144897
Epoch: 8, Loss: 1.7882169485092163
Epoch: 9, Loss: 1.7419979572296143
Epoch: 10, Loss: 1.7049269676208496
Epoch: 11, Loss: 1.6690196990966797
Epoch: 12, Loss: 1.6418002843856812
Epoch: 13, Loss: 1.6147176027297974
Epoch: 14, Loss: 1.5906462669372559
Epoch: 15, Loss: 1.5802345275878906
Epoch: 16, Loss: 1.5607959032058716
Epoch: 17, Loss: 1.5389306545257568
Epoch: 18, Loss: 1.518274188041687
Epoch: 19, Loss: 1.5056853294372559
Epoch: 20, Loss: 1.4942318201065063
Epoch: 21, Loss: 1.4834010601043701
Epoch: 22, Loss: 1.4771897792816162
Epoch: 23, Loss: 1.4641773700714111
Epoch: 24, Loss: 1.4565809965133667
Epoch: 25, Loss: 1.448169469833374
Epoch: 26, Loss: 1.442571997642517
Epoch: 27, Loss: 1.436651825904

# Saving Parameters

In [ ]:
# Saving data in Pickle to save model
with open('assignment5_params', 'wb') as handle:
    pickle.dump(params, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Loading saved parameters

In [ ]:
import pickle
loaded_model = pickle.load(open("assignment5_params", 'rb'))

# Generating Language

In [ ]:
# assigning saved parameters 
parameters = {}
parameters['W_xh'] = loaded_model['W_xh']
parameters['W_hh'] = loaded_model['W_hh']
parameters['W_ho'] = loaded_model['W_ho']
parameters['b_h'] = loaded_model['b_h']
parameters['b_o'] = loaded_model['b_o']

In [ ]:
ch = 't'  
i = 0
for key,val in vocab.items():
  if key == ch:
    i = val
# print(i)
# convert to one hot vector
input_ch = tf.one_hot(i,depth=vocab_size)
input_ch = tf.expand_dims(input_ch,axis=1)
input_ch = tf.transpose(input_ch)
print(input_ch.shape,W_xh.shape)

(1, 68) (512, 68)


In [ ]:
h_n =  tf.zeros((parameters['W_hh'].shape[0], 1))
character_index_list = [i]
for time_step in range(200):
     h_next, logits, out_t = rnn_cell(h_n, input_ch, parameters)
    #  print(out_t)
     output_ch = tf.one_hot(tf.math.argmax(out_t,axis=1),depth=68)
     h_n = h_next
     input_ch = output_ch
     index_value = tf.math.argmax(out_t,axis=1)
     character_index_list.append(index_value.numpy()[0])
# character_index_list

In [ ]:
seq = [ind_to_ch[ind] for ind in character_index_list]
print("".join(seq))

the hath stand a cardinal,
And there is no more than the world to the court,
And the better than the world to the court.

SIR TOBY BELCH:
And there is no more than the world to the court,
And the bette
